<img src="https://raw.githubusercontent.com/cebirnie92/KAUST-Iraya_SummerSchool2021/main/figures/logo.jpg">



# DAY THREE, Task(2): Title Generation

In this tutorial we will learn how to generate a title for a given paper's abstract. Title generation is a special type of text summarization, which is a well-known challenge of NLP.





# Problem Defenition
Given an abstract, the task is to generate a title for this paper.
### Example:
<a href="https://ibb.co/12CZLxn"><img src="https://i.ibb.co/WKrn613/Example.png" alt="Example" border="0"></a>"

In this tutorial, we will generate titles based on the BERT model, which is one of highly powerful pre-trained models for solving Natural Language Processing problems. We will use a python library named [**Headliner**](https://as-ideas.github.io/headliner/) that includes BERT model for text generation. 


 

# Headliner Library


**Headliner** is a sequence2sequence modeling library with the following features: 
*   It provides an easy-to-use API for both training and deployment. You can very easily use it in a few lines of code. 

*   It supports different encoder-decoder models, such as: 
  *   basic encoder-decoder
  *   encoder-decoder with attention
  *   transformer
  *   pre-trained BERT-model.


<img src="https://miro.medium.com/max/1826/1*MqFZFkCBd9N-yLLC75awZA.jpeg">

Figure: Encoder-decoder sequence-to-sequence model.

Source: https://as-ideas.github.io/headliner/


## **So, let’s have fun and try to generate titles!**


 



## Notebook Setup
To run this tutorial, first make sure to use GPU. 

If you are running on Colab please select Runtime--> change runtime  type--> GPU

**Required libraries and packages:**

If you are using your local machine, please install all these requirenments (provided in the environment.yml on github) (if you are using Colab, please jump to the next part)

*   pandas : https://pandas.pydata.org/
*   scikit-learn: https://scikit-learn.org/stable/

*   tenserflow-gpu: pip install tensorflow-gpu
*   tensorflow_datasets: pip install tensorflow_datasets



*   [headliner library](https://github.com/as-ideas/headliner ): pip install headliner








### Download all needed packages

In [ ]:
## Uncomment if running on CoLab
# # Install TensorFlow and library package via PyPI
# !pip install tensorflow-gpu
# !pip install headliner
# !pip install tensorflow_datasets

     |████████████████████████████████| 454.3MB 36kB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 2.3MB 38.4MB/s 
     |████████████████████████████████| 3.3MB 47.5MB/s 
     |████████████████████████████████| 901kB 39.6MB/s 


## Import the data
**Working on CoLab:**
Please add the file 'paper_info.csv' to your Drive

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
## Uncomment if running on CoLab
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Data directory
data_dir = "../data/" 
#data_dir = '/content/drive/My Drive/'  # Uncomment if running on CoLab

#read dataset file
df = pd.read_csv(data_dir+'paper_info.csv')
df.head()

,title,abstract
0,An algorithm to compute the full set of many-t...,The paper proposes an algorithm to compute the...
1,Relative entropy maximization and directed dif...,Motivated by the concept of maximum entropy me...
2,On the secant varieties to the tangential vari...,We study the dimensions of the higher secant v...
3,"The $L(2,1)$-Labeling Problem on Graphs",1. Introduction. The channel assignment probl...
4,Probabilities of causation: Bounds and identif...,This paper deals with the problem of estimati...


In [ ]:
#check how many examples we have in our dataset
len(df)

1517

## Clean the data

In [ ]:
# drop nan values
df = df.dropna()
# resetting the index
df = df.reset_index(drop=True)
df.head()

,title,abstract
0,An algorithm to compute the full set of many-t...,The paper proposes an algorithm to compute the...
1,Relative entropy maximization and directed dif...,Motivated by the concept of maximum entropy me...
2,On the secant varieties to the tangential vari...,We study the dimensions of the higher secant v...
3,"The $L(2,1)$-Labeling Problem on Graphs",1. Introduction. The channel assignment probl...
4,Probabilities of causation: Bounds and identif...,This paper deals with the problem of estimati...


In [ ]:
# remove noise from the text, remove new lines, tabs.
for i in range(0,len(df['abstract'])):
    if type(df['abstract'][i]) == str:
        df['abstract'][i] = df['abstract'][i].replace('\n','').replace('\r','').replace('/','')
        df['title'][i] = df['title'][i].replace('\n','').replace('\r','').replace('/','')
    else:
        print(str(df['abstract'][i]))
df.head()

,title,abstract
0,An algorithm to compute the full set of many-t...,The paper proposes an algorithm to compute the...
1,Relative entropy maximization and directed dif...,Motivated by the concept of maximum entropy me...
2,On the secant varieties to the tangential vari...,We study the dimensions of the higher secant v...
3,"The $L(2,1)$-Labeling Problem on Graphs",1. Introduction. The channel assignment probl...
4,Probabilities of causation: Bounds and identif...,This paper deals with the problem of estimati...


In [ ]:
len(df)

1517

## Split the data into training and testing sets

In [ ]:
#Split data into train, test
import sklearn.model_selection as model_selection
train_df, test_df = model_selection.train_test_split(df, test_size = 0.2)

print(len(test_df))
print(len(train_df))

304
1213


In [ ]:
#arrange the training data into list
train_data = []
for i in range(0,len(train_df['abstract'])):
    train_data.append((df['abstract'][i], df['title'][i]))

## Preprocessing

In this part we need to add BERT-specific start and end token to our training data using bert_preprocessor and spaCy packages

Here, the tokens **[CLS]** and **[SEP]** will be added to the start and end of each sentence.



In [ ]:
from headliner.preprocessing.bert_preprocessor import BertPreprocessor
from spacy.lang.en import English

# use BERT-specific start and end token
preprocessor = BertPreprocessor(nlp=English())
train_prep = [preprocessor(t) for t in train_data]
targets_prep = [t[1] for t in train_prep]

In [ ]:
#Check one sample of the abstracts after preprocessing
#it Should add the tokens before and and after each sentence.
train_prep[0][0]

'[CLS] The paper proposes an algorithm to compute the full set of many-to-many stable matchings when agents have substitutable preferences. [SEP] [CLS] The algorithm starts by calculating the two optimal stable matchings using the deferred-acceptance algorithm. [SEP] [CLS] Then, it computes each remaining stable matching as the firm-optimal stable matching corresponding to a new preference profile, which is obtained after modifying the preferences of a previously identified sequence of firms. [SEP] [CLS] © 2003 Elsevier B.V. All rights reserved. [SEP]'

In [ ]:
#Check one sample of the titles after preprocessing
targets_prep[2]

'[CLS] On the secant varieties to the tangential varieties of a Veronesean [SEP]'

## Training 

Here is the main part of the tutorial . We define the model and train it.

You can play with different parameters to see how the model works. 

In [ ]:
# Define the model and train it
# You need to be quite patient, since the model has a lot of params
import tensorflow_datasets as tfds
from transformers import BertTokenizer
from headliner.model.bert_summarizer import BertSummarizer
from headliner.preprocessing.bert_vectorizer import BertVectorizer
from headliner.trainer import Trainer

# Use a pre-trained BERT embedding and BERT tokenizer for the encoder 
tokenizer_input = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer_target = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    targets_prep, target_vocab_size=2**13,  reserved_tokens=[preprocessor.start_token, preprocessor.end_token])


vectorizer = BertVectorizer(tokenizer_input, tokenizer_target)

#The parameters of the model
summarizer = BertSummarizer(num_heads=2,
                            feed_forward_dim=512,
                            num_layers_encoder=0,
                            num_layers_decoder=4,
                            bert_embedding_encoder='bert-base-uncased',
                            embedding_size_encoder=768,
                            embedding_size_decoder=768,
                            dropout_rate=0.1,
                            max_prediction_len=80)
summarizer.init_model(preprocessor, vectorizer)

trainer = Trainer(batch_size=2)
trainer.train(summarizer, train_data, num_epochs=10)

INFO:absl:SubwordTextEncoder build: trying min_token_count 607
INFO:absl:SubwordTextEncoder build: trying min_token_count 303
INFO:absl:SubwordTextEncoder build: trying min_token_count 151
INFO:absl:SubwordTextEncoder build: trying min_token_count 75
INFO:absl:SubwordTextEncoder build: trying min_token_count 37
INFO:absl:SubwordTextEncoder build: trying min_token_count 18
INFO:absl:SubwordTextEncoder build: trying min_token_count 9
INFO:absl:SubwordTextEncoder build: trying min_token_count 4
INFO:absl:SubwordTextEncoder build: trying min_token_count 2
INFO:absl:SubwordTextEncoder build: trying min_token_count 1
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expect

training an already initialized model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


epoch 0, batch 10, logs: {'loss': 7.167372131347657}
epoch 0, batch 20, logs: {'loss': 7.1193702697753904}
epoch 0, batch 30, logs: {'loss': 7.256068420410156}
epoch 0, batch 40, logs: {'loss': 7.254052734375}
epoch 0, batch 50, logs: {'loss': 7.2374853515625}
epoch 0, batch 60, logs: {'loss': 7.250812784830729}
epoch 0, batch 70, logs: {'loss': 7.141945975167411}
epoch 0, batch 80, logs: {'loss': 7.1068885803222654}
epoch 0, batch 90, logs: {'loss': 7.070761447482639}
epoch 0, batch 100, logs: {'loss': 7.04937744140625}
epoch 0, batch 110, logs: {'loss': 7.0407936789772725}
epoch 0, batch 120, logs: {'loss': 7.005577087402344}
epoch 0, batch 130, logs: {'loss': 6.99504159780649}
epoch 0, batch 140, logs: {'loss': 6.9932861328125}
epoch 0, batch 150, logs: {'loss': 6.960576171875}
epoch 0, batch 160, logs: {'loss': 6.9315338134765625}
epoch 0, batch 170, logs: {'loss': 6.921579877068015}
epoch 0, batch 180, logs: {'loss': 6.918975151909722}
epoch 0, batch 190, logs: {'loss': 6.90308195

epoch 3, batch 1510, logs: {'loss': 5.545142364501953}
epoch 3, batch 1520, logs: {'loss': 5.417619705200195}
epoch 3, batch 1530, logs: {'loss': 5.433860270182292}
epoch 3, batch 1540, logs: {'loss': 5.375527191162109}
epoch 3, batch 1550, logs: {'loss': 5.350341796875}
epoch 3, batch 1560, logs: {'loss': 5.3614501953125}
epoch 3, batch 1570, logs: {'loss': 5.371289498465401}
epoch 3, batch 1580, logs: {'loss': 5.392351150512695}
epoch 3, batch 1590, logs: {'loss': 5.437780083550347}
epoch 3, batch 1600, logs: {'loss': 5.455440063476562}
epoch 3, batch 1610, logs: {'loss': 5.442047119140625}
epoch 3, batch 1620, logs: {'loss': 5.442609151204427}
epoch 3, batch 1630, logs: {'loss': 5.433273550180289}
epoch 3, batch 1640, logs: {'loss': 5.441652570452009}
epoch 3, batch 1650, logs: {'loss': 5.459882405598958}
epoch 3, batch 1660, logs: {'loss': 5.435660934448242}
epoch 3, batch 1670, logs: {'loss': 5.423834587545956}
epoch 3, batch 1680, logs: {'loss': 5.426718139648438}
epoch 3, batch 

epoch 5, batch 2990, logs: {'loss': 4.658330676020408}
epoch 5, batch 3000, logs: {'loss': 4.66214404296875}
epoch 6, batch 3010, logs: {'loss': 4.239145278930664}
epoch 6, batch 3020, logs: {'loss': 4.536834335327148}
epoch 6, batch 3030, logs: {'loss': 4.591202799479166}
finished iterating over dataset, total batches: 3030
epoch 6, batch 3040, logs: {'loss': 4.4915412902832035}
epoch 6, batch 3050, logs: {'loss': 4.346260986328125}
epoch 6, batch 3060, logs: {'loss': 4.322457377115885}
epoch 6, batch 3070, logs: {'loss': 4.273846435546875}
epoch 6, batch 3080, logs: {'loss': 4.261737823486328}
epoch 6, batch 3090, logs: {'loss': 4.247124565972222}
epoch 6, batch 3100, logs: {'loss': 4.255319213867187}
epoch 6, batch 3110, logs: {'loss': 4.243368530273438}
epoch 6, batch 3120, logs: {'loss': 4.2493441263834635}
epoch 6, batch 3130, logs: {'loss': 4.227216045673077}
epoch 6, batch 3140, logs: {'loss': 4.212058367047991}
epoch 6, batch 3150, logs: {'loss': 4.183502604166667}
epoch 6, ba

epoch 8, batch 4450, logs: {'loss': 3.4294694010416666}
epoch 8, batch 4460, logs: {'loss': 3.4254654594089673}
epoch 8, batch 4470, logs: {'loss': 3.4163789789727392}
epoch 8, batch 4480, logs: {'loss': 3.4081581115722654}
epoch 8, batch 4490, logs: {'loss': 3.3993806800063777}
epoch 8, batch 4500, logs: {'loss': 3.389989501953125}
epoch 9, batch 4510, logs: {'loss': 3.1116186141967774}
epoch 9, batch 4520, logs: {'loss': 3.217171478271484}
epoch 9, batch 4530, logs: {'loss': 3.205359141031901}
epoch 9, batch 4540, logs: {'loss': 3.128487968444824}
epoch 9, batch 4550, logs: {'loss': 3.1015313720703124}
epoch 9, batch 4560, logs: {'loss': 3.0595672607421873}
epoch 9, batch 4570, logs: {'loss': 3.06434326171875}
epoch 9, batch 4580, logs: {'loss': 3.0796680450439453}
epoch 9, batch 4590, logs: {'loss': 3.0890801323784722}
epoch 9, batch 4600, logs: {'loss': 3.0751828002929686}
epoch 9, batch 4610, logs: {'loss': 3.099008733575994}
epoch 9, batch 4620, logs: {'loss': 3.0973670959472654}

### Test the model

Let's test the model with some data from our testing set.

We will check the ground truth titles with the generated ones.

In [ ]:
#select some samples from the test set
sample_test_set=test_df[:10]
sample_test_set = sample_test_set.reset_index(drop=True)

In [ ]:
#10 epochs - batch=2
for i in range(0, len(sample_test_set['abstract'])):
    #predict the title given the abstract
    prediction = summarizer.predict(sample_test_set['abstract'][i])

    #print the ground-truth title  
    print('t:',sample_test_set['title'][i])
    #print the predicted title
    print('p:',prediction)
    print('')

t: Positive definite matrices and the S-divergence
p: The approximation of the smoothing problem [SEP]

t: Anasazi software for the numerical solution of large-scale eigenvalue problems
p: Algorithm 782: codes for the solution of large-scale eigenvalue problems [SEP]

t: Approximate Qualitative Temporal Reasoning
p: Comparing multiagent systems and applications [SEP]

t: Towards Mathematical Knowledge Management for Electrical Engineering
p: A new approach for the Complexity of Mathematical Knowledge [SEP]

t: On the level sets of a distance function in a Minkowski space
p: On a Convex Optimization [SEP]

t: The complexity of Markov decision processes
p: The complexity of a Markov Decision Processes [SEP]

t: Network-Based Drug Ranking and Repositioning with Respect to DrugBank Therapeutic Categories
p: A new approach for the Classification of Brain -based based based on Protein Interaction [SEP]

t: Mining association rules among gene functions in clusters of similar gene expression m

## ToDo Task





1.   Test the model using geoscience data used in the previous tutorials.
  1.   Read the abstracts into a list. 
  2.   Apply the testing part of this tutorial on this list.
2.   Change some parameters of the model, like the epochs number and the batch size, and see what is the impact of these parameters.
















## References:
*   https://as-ideas.github.io/headliner/
*   https://medium.com/axel-springer-tech/headliner-easy-training-and-deployment-of-seq2seq-models-2a26508b4dae





